# Predict Tweet Toxicity

In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
filename = "twitter_data/andrea_scraped/electoralcollege2016"
# filename1 = "twitter_data/andrea_scraped/abortion2016"
# filename2 = "twitter_data/andrea_scraped/prochoice2016"
# filename3 = "twitter_data/andrea_scraped/prolife2016"

In [3]:
df = pd.read_csv(filename, encoding = "ISO-8859-1",
                 names = ['date', 'fullname', 'id', 'likes', 'replies', 
                            'retweets', 'text', 'url', 'user', 'month'],
                engine='python')

# df1 = pd.read_csv(filename1, encoding = "ISO-8859-1",
#                  names = ['date', 'fullname', 'id', 'likes', 'replies', 
#                             'retweets', 'text', 'url', 'user', 'month'])

# df2 = pd.read_csv(filename2, encoding = "ISO-8859-1",
#                  names = ['date', 'fullname', 'id', 'likes', 'replies', 
#                             'retweets', 'text', 'url', 'user', 'month'])

# df3 = pd.read_csv(filename3, encoding = "ISO-8859-1",
#                  names = ['date', 'fullname', 'id', 'likes', 'replies', 
#                             'retweets', 'text', 'url', 'user', 'month'])

# df = pd.concat([df1, df2, df3])
df = df.drop_duplicates()
df = df.dropna()
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [4]:
for date in df.sort_values('date').date.unique():
    print(date)

2016-01-01 20:03:01
2016-01-01 20:10:54
2016-01-01 20:15:06
2016-01-01 20:25:17
2016-01-01 20:40:46
2016-01-01 20:45:36
2016-01-01 20:51:02
2016-01-01 20:54:05
2016-01-01 20:56:06
2016-01-01 21:01:02
2016-01-01 21:06:59
2016-01-01 22:00:12
2016-01-01 22:14:29
2016-01-01 22:14:31
2016-01-01 22:39:30
2016-01-01 22:42:39
2016-01-01 22:54:04
2016-01-01 22:59:10
2016-01-01 23:19:01
2016-01-01 23:39:46
2016-01-02 19:48:22
2016-01-02 20:07:43
2016-01-02 20:10:29
2016-01-02 20:10:30
2016-01-02 20:13:02
2016-01-02 20:23:49
2016-01-02 20:39:07
2016-01-02 20:41:02
2016-01-02 20:54:13
2016-01-02 20:54:39
2016-01-02 21:03:44
2016-01-02 21:11:06
2016-01-02 21:16:05
2016-01-02 21:18:26
2016-01-02 21:24:05
2016-01-02 21:24:16
2016-01-02 21:27:54
2016-01-02 21:28:10
2016-01-02 22:50:16
2016-01-02 23:06:05
2016-01-03 16:35:24
2016-01-03 16:49:55
2016-01-03 17:03:57
2016-01-03 17:11:18
2016-01-03 18:05:51
2016-01-03 18:18:34
2016-01-03 19:12:02
2016-01-03 19:15:42
2016-01-03 19:19:47
2016-01-03 19:40:38


2016-02-02 23:51:47
2016-02-02 23:52:16
2016-02-02 23:53:36
2016-02-02 23:54:43
2016-02-02 23:55:14
2016-02-02 23:57:01
2016-02-02 23:57:13
2016-02-02 23:57:18
2016-02-03 23:10:08
2016-02-03 23:10:10
2016-02-03 23:15:00
2016-02-03 23:15:01
2016-02-03 23:16:35
2016-02-03 23:20:11
2016-02-03 23:22:05
2016-02-03 23:25:35
2016-02-03 23:32:30
2016-02-03 23:34:48
2016-02-03 23:35:37
2016-02-03 23:35:43
2016-02-03 23:36:01
2016-02-03 23:39:43
2016-02-03 23:40:09
2016-02-03 23:45:35
2016-02-03 23:52:37
2016-02-03 23:52:51
2016-02-03 23:58:07
2016-02-03 23:58:14
2016-02-04 22:15:42
2016-02-04 22:39:07
2016-02-04 22:40:22
2016-02-04 22:48:24
2016-02-04 22:52:21
2016-02-04 22:57:47
2016-02-04 22:59:18
2016-02-04 23:00:44
2016-02-04 23:04:57
2016-02-04 23:06:02
2016-02-04 23:12:55
2016-02-04 23:13:09
2016-02-04 23:14:06
2016-02-04 23:16:02
2016-02-04 23:20:35
2016-02-04 23:23:26
2016-02-04 23:24:06
2016-02-04 23:30:03
2016-02-04 23:32:05
2016-02-04 23:36:48
2016-02-05 20:16:41
2016-02-05 20:42:18


2016-04-19 23:52:56
2016-04-19 23:54:01
2016-04-19 23:55:12
2016-04-19 23:56:25
2016-04-19 23:56:48
2016-04-19 23:57:51
2016-04-19 23:59:13
2016-04-20 23:27:49
2016-04-20 23:31:21
2016-04-20 23:31:30
2016-04-20 23:32:50
2016-04-20 23:34:56
2016-04-20 23:35:26
2016-04-20 23:37:08
2016-04-20 23:40:06
2016-04-20 23:44:08
2016-04-20 23:44:25
2016-04-20 23:44:33
2016-04-20 23:44:45
2016-04-20 23:45:12
2016-04-20 23:51:13
2016-04-20 23:53:02
2016-04-20 23:53:06
2016-04-20 23:55:37
2016-04-20 23:57:04
2016-04-20 23:57:16
2016-04-20 23:57:52
2016-04-21 22:34:15
2016-04-21 22:40:51
2016-04-21 22:50:19
2016-04-21 22:55:18
2016-04-21 22:57:01
2016-04-21 22:59:50
2016-04-21 23:01:44
2016-04-21 23:03:00
2016-04-21 23:05:06
2016-04-21 23:08:14
2016-04-21 23:12:33
2016-04-21 23:20:48
2016-04-21 23:21:27
2016-04-21 23:28:19
2016-04-21 23:28:26
2016-04-21 23:33:30
2016-04-21 23:42:48
2016-04-21 23:44:36
2016-04-21 23:44:57
2016-04-21 23:49:17
2016-04-22 21:45:16
2016-04-22 21:53:48
2016-04-22 22:08:48


2016-07-02 23:54:48
2016-07-03 21:41:37
2016-07-03 21:41:52
2016-07-03 21:46:45
2016-07-03 21:46:56
2016-07-03 21:51:43
2016-07-03 21:59:20
2016-07-03 22:20:02
2016-07-03 22:40:33
2016-07-03 22:40:51
2016-07-03 22:50:06
2016-07-03 22:57:14
2016-07-03 23:27:09
2016-07-03 23:34:53
2016-07-03 23:41:12
2016-07-03 23:45:44
2016-07-03 23:46:15
2016-07-03 23:50:49
2016-07-03 23:52:09
2016-07-03 23:53:38
2016-07-03 23:54:34
2016-07-04 21:03:27
2016-07-04 21:04:35
2016-07-04 21:11:20
2016-07-04 21:22:43
2016-07-04 21:39:27
2016-07-04 21:45:26
2016-07-04 21:47:06
2016-07-04 22:02:44
2016-07-04 22:21:20
2016-07-04 22:27:54
2016-07-04 22:30:36
2016-07-04 22:37:55
2016-07-04 22:46:02
2016-07-04 22:49:46
2016-07-04 22:50:23
2016-07-04 22:51:43
2016-07-04 23:10:16
2016-07-04 23:18:46
2016-07-04 23:50:37
2016-07-04 23:59:12
2016-07-05 22:45:06
2016-07-05 22:50:01
2016-07-05 22:50:26
2016-07-05 22:58:26
2016-07-05 22:59:34
2016-07-05 23:01:13
2016-07-05 23:02:12
2016-07-05 23:09:09
2016-07-05 23:24:20


2016-09-11 22:56:22
2016-09-11 23:05:14
2016-09-11 23:11:01
2016-09-11 23:15:54
2016-09-11 23:19:32
2016-09-11 23:26:00
2016-09-11 23:34:28
2016-09-11 23:38:09
2016-09-11 23:42:54
2016-09-11 23:43:06
2016-09-11 23:48:36
2016-09-11 23:49:26
2016-09-11 23:49:52
2016-09-11 23:51:31
2016-09-11 23:57:58
2016-09-12 23:12:13
2016-09-12 23:17:06
2016-09-12 23:25:12
2016-09-12 23:29:08
2016-09-12 23:30:07
2016-09-12 23:30:57
2016-09-12 23:32:05
2016-09-12 23:32:09
2016-09-12 23:35:23
2016-09-12 23:37:06
2016-09-12 23:38:19
2016-09-12 23:40:07
2016-09-12 23:42:46
2016-09-12 23:47:55
2016-09-12 23:48:35
2016-09-12 23:50:52
2016-09-12 23:51:26
2016-09-12 23:53:12
2016-09-12 23:58:10
2016-09-13 22:29:59
2016-09-13 22:32:27
2016-09-13 22:40:54
2016-09-13 22:45:44
2016-09-13 22:51:13
2016-09-13 22:53:05
2016-09-13 22:55:06
2016-09-13 22:57:19
2016-09-13 23:06:56
2016-09-13 23:09:41
2016-09-13 23:10:14
2016-09-13 23:13:15
2016-09-13 23:15:16
2016-09-13 23:15:59
2016-09-13 23:16:23
2016-09-13 23:21:05


2016-11-30 23:59:07
2016-11-30 23:59:11
2016-11-30 23:59:13
2016-11-30 23:59:16
2016-11-30 23:59:19
2016-11-30 23:59:31
2016-11-30 23:59:33
2016-11-30 23:59:36
2016-11-30 23:59:40
2016-11-30 23:59:45
2016-11-30 23:59:49
2016-11-30 23:59:53
2016-12-01 15:43:38
2016-12-01 23:52:43
2016-12-01 23:56:52
2016-12-01 23:57:25
2016-12-01 23:57:57
2016-12-01 23:58:06
2016-12-01 23:58:36
2016-12-01 23:58:38
2016-12-01 23:58:46
2016-12-01 23:58:54
2016-12-01 23:58:56
2016-12-01 23:59:17
2016-12-01 23:59:19
2016-12-01 23:59:21
2016-12-01 23:59:25
2016-12-01 23:59:27
2016-12-01 23:59:31
2016-12-01 23:59:39
2016-12-01 23:59:44
2016-12-01 23:59:59
2016-12-02 23:16:58
2016-12-02 23:58:26
2016-12-02 23:58:28
2016-12-02 23:58:35
2016-12-02 23:58:40
2016-12-02 23:58:46
2016-12-02 23:58:48
2016-12-02 23:58:56
2016-12-02 23:59:01
2016-12-02 23:59:18
2016-12-02 23:59:19
2016-12-02 23:59:22
2016-12-02 23:59:46
2016-12-02 23:59:55
2016-12-03 23:56:54
2016-12-03 23:57:01
2016-12-03 23:57:09
2016-12-03 23:57:28


In [5]:
len(df)

7215

### Vectorize tweets and predict toxicity with model

In [6]:
# Load vectorizers
word_vectorizer = pickle.load(open("models/word_vectorizer.pickle", "rb" ))
char_vectorizer = pickle.load(open("models/char_vectorizer.pickle", "rb" ))

In [7]:
# Vectorize tweets
tweet_word_features = word_vectorizer.transform(df['text'])
tweet_char_features = char_vectorizer.transform(df['text'])
tweet_features = hstack([tweet_word_features, tweet_char_features])

In [8]:
# Load models
loaded_models = {}
for col in target_columns:
    loaded_models[col] = pickle.load(open('models/model_{}.sav'.format(col), "rb" ))

In [9]:
# Run models
for col in target_columns:
    df[col] = loaded_models[col].predict_proba(tweet_features)[:, 1]

In [10]:
df.head()

,date,fullname,id,likes,replies,retweets,text,url,user,month,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,2016-07-19 23:57:00,Trill Millennial,755552048901267456,0.0,1.0,0.0,I'm giving Trump a 5% chance. The electoral co...,/TrillMillN7/status/755552048901267456,TrillMillN7,2016-07-01,0.136902,0.002493,0.001714,0.000315,0.000220,0.000086
1,2016-07-19 23:55:25,(((Shining City))),755551649024897024,0.0,0.0,0.0,"Michael, you do understand the Electoral Colle...",/shiningcity1776/status/755551649024897024,shiningcity1776,2016-07-01,0.000923,0.001970,0.007655,0.000027,0.001057,0.000072
2,2016-07-19 23:55:09,Chuck Daniels,755551583518068737,0.0,0.0,1.0,Can't wait for Trump to insinuate that the who...,/chuck_daniels_/status/755551583518068737,chuck_daniels_,2016-07-01,0.002701,0.000595,0.001825,0.000019,0.002100,0.000015
3,2016-07-19 23:54:43,Hobiecat,755551475670081536,1.0,0.0,0.0,If the electoral college doesn't rig it for HR...,/PANYGAFLgirl/status/755551475670081536,PANYGAFLgirl,2016-07-01,0.002702,0.003599,0.002964,0.000046,0.001603,0.000124
4,2016-07-19 23:54:30,Marc Lippincott Â ðºð¸,755551419680239616,0.0,1.0,0.0,Is that English? And I'm not worried about HRC...,/marclippincott/status/755551419680239616,marclippincott,2016-07-01,0.001951,0.001348,0.000121,0.000037,0.000378,0.000021


In [11]:
df['classification'] = df[target_columns].idxmax(axis=1)

In [12]:
for row in df.index[:10]:
    print(df.text[row])
    print(df.classification[row])

0    I'm giving Trump a 5% chance. The electoral co...
0    Let Johnson Stein Debate.Tossed out lawsuit is...
0    As the race tightens, don't expect the Elector...
0    @debates how about including @GovGaryJohnson a...
0    Gore: We Should Abolish Electoral College, Pop...
0    Next on @MSNBC:\nPollyBaca - a Democratic elec...
0    Electoral College Prediction On Clinton Vs Tru...
0    .@GuruBluXVIII Again, if no candidate gets 270...
0    Both PoliticalParties & ElectoralCollege are a...
0    I would wager that not even 40% of Americans c...
0    Been doing research on "superdelegates" - my c...
0    If no one else stops Donald Trump, the Elector...
0    Trump is now even or ahead of Crooked Hillary ...
0    Tell me what states Jill Steun will win. Tell ...
0    why is there still an #electoral college. Do t...
0    a good percentage of people dont even know how...
0    ã@jslagra The real candidates in Presidentia...
0    @trumpflorida16  keep a eye on the Electoral c...
0    If ac

In [13]:
# export df
export_filename = "classified/Andrea/electoralcollege2016"
df.to_csv(export_filename)

### Visualizations

In [14]:
# visualize obscenity
df.groupby('month')['obscene'].mean().plot()

In [15]:
df['likes'] = df['likes'].astype(float)

# normalizing identity hate by tweet popularity
df['popular_obscene'] = df['obscene']*df['likes']
df.groupby('month')['popular_obscene'].mean().plot()

In [16]:
# normalizing identity hate by number of popular tweets
(df.groupby('month')['popular_obscene'].sum() / df.groupby('month')['likes'].sum()).plot()

In [17]:
# normalizing popular tweets by number of popular tweets
(df.groupby('month')['popular_obscene'].sum() / df.groupby('month')['likes'].count()).plot()